# Imports

In [1]:
import os

import time
import torch
import random

import warnings
import numpy as np
import pandas as pd
from PIL import Image
from glob import glob

from json import loads, dumps

import seaborn as sns 

import typing_extensions as typing
from google.generativeai.types import HarmCategory, HarmBlockThreshold

from collections import Counter

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix,classification_report 


# Data Loading

In [2]:
df = pd.read_csv('/kaggle/input/youtube-data/all_unique_codes3.csv')
df

,Video link,Primary Tag,Secondary Tag,Rating,Description,Available?
0,Wma_iVqKRWw,inappropriate,Scariness,Mild,"lyric video, fire, forest burning, explosion",Yes
1,1dtzSRlfBDk,child directed,Educational Content,NaN,ad about a Math and logic based visual game. ...,Yes
2,5KRcG9gVhio,inappropriate,Offensive language,Extreme,spanish mv; lyrics inappropriate,Yes
3,1EYvCnQj1uQ,inappropriate,Offensive language,Extreme,spanish mv; inapp lyrics,Yes
4,hkworAjntAI,inappropriate,Scariness,Moderate,"song about stopping domestic abuse, yelling, b...",Yes
...,...,...,...,...,...,...
2639,eRaJzy7dtTk,child directed,Cartoons,NaN,ad for a compilation of the cartoon baby alive,Yes
2640,BqHBzimyRbM,irrelevant,Home and Lifestyle,NaN,Ad for a company that has deals on furnishing,Yes
2641,rYSTqeFRLo8,irrelevant,Gaming & Entertainment,NaN,Ad for a game to build settlements,Yes
2642,CllAJ6PkRIA,irrelevant,Health and Wellness,NaN,Ad for a medicinal supplement,Yes


In [3]:
available_ad_ids = os.listdir('/kaggle/input/youtube-data/Ads/Ads') 
len(available_ad_ids)

2520

In [4]:
all_ids = [] #list of video ids

In [ ]:
primary_labels = []
video_ids = []

for v in all_ids:
    try:
                    
        primary_tag = df.loc[df['Video link']==v,'Primary Tag'].values[0]
        
        if (primary_tag!="inaudible" or primary_tag!="excluded"):
            video_ids.append(v)
            primary_labels.append(primary_tag)
   
    except:
        pass

print("Reasonable video ids:",len(video_ids))



**Defining Model** 

In [6]:
api_key = "" 

In [7]:
import google.generativeai as genai 
os.environ["API_KEY"] = api_key 
genai.configure(api_key=os.environ["API_KEY"]) 

In [8]:
model = genai.GenerativeModel("gemini-1.5-flash", system_instruction="You are an expert content moderator.") 

In [9]:
class LLM_Output(typing.TypedDict):
    label: str
    language: list[str]
    response: str

In [10]:
prompt = """A video can be considered inappropriate for children if it contains violent or graphic content (cartoonish, realistic, inclusion of gore and blood),
  self-harm or suicide,
  elements of horror or creepy themes,
  loud and violent sounds,
  zombies, skeletons, scary masks, scary clowns, fire in dangerous settings,
  extremely distressing scenes (such as car crashes),
  extreme stunts (depictions of life endangering or high-risk physical activities),
  fight sports (such as boxing, wrestling, martial arts),
  first-person shooter and battle games (games intended for children over 13 should be considered inapproprite),
  weapons (such as firearms, explosives, swords, axes etc.),
  sexual content (innuendos, overt sexual behavior, nudity, sexual themes, sexual suggestiveness etc.),
  vulgar dancing (such as twerking etc.),
  discrimination (based on race/ethnicity/gender/ability),
  profane or explicit language (from mild words such as "idiot" and "stupid" to severe swearing, hate speech or signs),
  drinking, drugs, smoking, gambling, crude humour,
  inappropriate clothing (with significant skin exposure),
  significant romantic intimacy.

  Remember that this criteria is not exhasutive.

  You have been given a video and it's audio.
  Based on both the visual content and audio, is the video and provided audio inappropriate?

  If it is, label it as "inappropriate".
  If it is not, label it as "appropriate".

  Justify your conclusion.

  Also, mention what language is being spoken if there is any.

  Make the outputs in JSON format.

  """

**Running Model on Dataset** 

In [ ]:
ids = []
predicted_labels = []
languages = []
responses = []
ground_truths = []
remaining = []

for i in range(0, len(video_ids)): 

        
    
    if (video_ids[i] in available_ad_ids):

        try:
            contents_of_ad = os.listdir('/kaggle/input/youtube-data/Ads/Ads/' + video_ids[i]) 
            contents_of_ad.remove('audio.mp3') 
            path_to_video = '/kaggle/input/youtube-data/Ads/Ads/' + video_ids[i] + '/' + contents_of_ad[0] 
            path_to_audio = '/kaggle/input/youtube-data/Ads/Ads/' + video_ids[i] + '/audio.mp3' 

            ## upload video 
            video_file = genai.upload_file(path=path_to_video) 
    
            ## upload audio 
            audio_file = genai.upload_file(path=path_to_audio) 
    
            ## check if video has uploaded - wait for it to upload
            while video_file.state.name == "PROCESSING":
              print('.', end='')
              time.sleep(10)
              video_file = genai.get_file(video_file.name)
    
            if video_file.state.name == "FAILED":
              raise ValueError(video_file.state.name)

            try: 
                response = model.generate_content([audio_file,video_file, prompt],
                                    generation_config=genai.GenerationConfig(
                                        response_mime_type="application/json", response_schema=LLM_Output,temperature=0.0),

                                    safety_settings={
                                      HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
                                      HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
                                      HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
                                      HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE
                                  })
            except Exception as e:
                print(f"Error making inference: {e}")
                remaining.append(video_ids[i])
                continue 

            # Wrap response.text access in try-except
            try:
                print("Completed for video number:", i, ' ', video_ids[i])

                dictionary = loads(response.text)
                print('True Label:', primary_labels[i], 'Response:', dictionary)

                ids.append(video_ids[i])
                predicted_labels.append(dictionary['label'])
                languages.append(dictionary['language'])
                responses.append(dictionary['response'])
                ground_truths.append(primary_labels[i]) 
                
            except Exception as e:
                print(f"Error processing response.text: {e}")
                remaining.append(video_ids[i])
                continue

        except Exception as e:
            print(f"Unexpected error: {e}")
            remaining.append(video_ids[i])
            continue
        
        time.sleep(15)

        

        

In [ ]:
# At the end, print remaining videos 

print("Remaining videos with errors:", remaining) 

In [ ]:
for i in range(len(responses)): 
    print('True Label: ', ground_truths[i], '\tPrediction: ', predicted_labels[i]) 

In [14]:
new_df = pd.DataFrame({
    'Video Id': ids,
    'Primary Label': ground_truths,
    'Predicted Label': predicted_labels,
    'Response': responses, 
    'Languages': languages 
})

new_df.head() 

,Video Id,Primary Label,Predicted Label,Response,Languages
0,B1z2h_tZZq0,inappropriate,inappropriate,The music video contains scenes of a man and a...,[Punjabi]
1,_axD_oF_PU8,irrelevant,appropriate,The video is an advertisement for Cash Plus. I...,[Arabic]
2,K77-yXo-0r0,irrelevant,appropriate,The video shows a compilation of beautiful nat...,[]
3,kHdUFZ16Yt0,irrelevant,appropriate,The video shows a compilation of beautiful nat...,[]
4,_BsFVKSGxI0,irrelevant,appropriate,The video shows a can of insecticide killing m...,[Bengali]


In [15]:
path_to_new_csv = ""
new_df.to_csv(path_to_new_csv, index=False)

In [16]:
# Changing to binary lists 

predictions = [1 if pred == 'inappropriate' else 0 for pred in predicted_labels] 
ground_truths = [1 if label == 'inappropriate' else 0 for label in ground_truths] 

In [ ]:

report = classification_report(ground_truths, predictions) 
print(report) 

In [ ]:

cm = confusion_matrix(ground_truths, predictions)

plt.figure(figsize=(6,4))
sns.heatmap(cm, annot=True, fmt='d', cmap='RdPu', xticklabels=['Appropriate', 'Inapproriate'], yticklabels=['Appropriate', 'Inapproriate'])
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()